# Example 29: Path Planning with Rapidly-Exploring Random Trees

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Rapidly-Exploring Random Trees](#ekf)
    * [Test case](#test_case)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="ackw"></a> Acknowledgements

Most of the content in this notebook is taken from wikipedia entry: <a href="https://en.wikipedia.org/wiki/Rapidly-exploring_random_tree">Rapidly-exploring random tree</a>.

## <a name="overview"></a> Overview

In this notebook we discuss Rapidly-Exploring Random Trees or RRTs for short. RRTs were developed by Steven M. LaValle and James J. Kuffner Jr.,  see  <a href="http://msl.cs.uiuc.edu/~lavalle/papers/Lav98c.pdf">Rapidly-Exploring Random Trees: A New Tool For Path Planning</a> and <a href="http://www.cs.columbia.edu/~allen/F15/NOTES/LavKuf01rrt.pdf">Rapidly-Exploring Random Trees: Progress and Prospects</a>. A simple demostration of RRTs can be found <a href="https://www.youtube.com/watch?v=Ob3BIJkQJEw">here</a>.

### <a name="ekf"></a> Rapidly-exploring random trees

RRTs can be viewed as a technique to generate open-loop trajectories for nonlinear systems with state constraints. An RRT can also be considered as a Monte-Carlo method to bias search into the largest Voronoi regions of a graph in a configuration space. 

An RRT grows a tree rooted at the starting configuration by using random samples from the search space. As each sample is drawn, a connection is attempted between it and the nearest state in the tree. If the connection is feasible (passes entirely through free space and obeys any constraints), this results in the addition of the new state to the tree. With uniform sampling of the search space, the probability of expanding an existing state is proportional to the size of its Voronoi region. As the largest Voronoi regions belong to the states on the frontier of the search, this means that the tree preferentially expands towards large unsearched areas. 

RRT growth can be biased by increasing the probability of sampling states from a specific area. Most practical implementations of RRTs make use of this to guide the search towards the planning problem goals. This is accomplished by introducing a small probability of sampling the goal to the state sampling procedure. The higher this probability, the more greedily the tree grows towards the goal. 

The algorithm, taken from <a href="http://msl.cs.uiuc.edu/~lavalle/papers/Lav98c.pdf">Rapidly-Exploring Random Trees: A New Tool For Path Planning</a>, describes the basic construction of an RRT

<figure>
<img src="basic_rrt.png" alt="Generate RRT algorithm"
	title="Generate RRT algorithm" width="500" height="450" />
<figcaption>Figure: Generate RRT algorithm. Image from Rapidly-Exploring Random Trees: A New Tool For Path Planning.</figcaption>
</figure>

### Test case

For the test case we will consider a square domain $[0,50]\times[0,50]$. The initial position of the agent is at $[25.0, 25.0]$. The test case will allow the tree to grow on the positive 45 degrees direction of the starting position. This is done by using the following dynamics function:

```
auto dynamics = [](const SysState<2>& s1, const SysState<2>& s2){
           SysState<2> s;
           s.set(0, {"X", 0.0});
           s.set(1, {"Y", 0.0});
           s["X"] = s1["X"] + 0.5;
           s["Y"] = s1["Y"] + 0.5;
           return std::make_tuple(s, DynVec<real_t>(2, 0.0));
        };
```

This will not produce the following tree (note that in order to plot the tree you need <a href="https://networkx.github.io/documentation/networkx-2.4/index.html">NetworkX</a> Python library) 

<img src="line_rrt.png"
     alt="Line RRT"
     style="float: left; margin-right: 10px; width: 500px;" />

Obviously, this is not something really exciting but it is a starting point for testing our code. Using the following dynamics: 

``` 
    auto dynamics = [](const SysState<2>& s1, const SysState<2>& s2){
       SysState<2> s;
       s.set(0, {"X", 0.0});
       s.set(1, {"Y", 0.0});
       s["X"] = s1["X"] + (s2["X"] - s1["X"]);
       s["Y"] = s1["Y"] + (s2["Y"] - s1["Y"]);

       return std::make_tuple(s, DynVec<real_t>(2, 0.0));
    };
```

We have the following tress with 50, 100 and 1000 nodes respectivelly

<img src="rrt_50.png"
     alt="RRT 50 nodes"
     style="float: left; margin-right: 10px; width: 500px;" />

<img src="rrt_100.png"
     alt="RRT 100 nodes"
     style="float: left; margin-right: 10px; width: 500px;" />

<img src="rrt_1000.png"
     alt="RRT 1000 nodes"
     style="float: left; margin-right: 10px; width: 500px;" />

In the second test case we will try to actually build a path using RRT. Again, we will use a square domain $[0,50]\times[0,50]$. The initial position of the agent is at $[25.0, 25.0]$. Now the dynamics is described by the following lambda function

```

auto dynamics = [](const SysState<2>& s1, const SysState<2>& s2){
           SysState<2> s;
           s.set(0, {"X", 0.0});
           s.set(1, {"Y", 0.0});
           s["X"] = s1["X"] + 0.5;
           s["Y"] = s1["Y"] + 0.5;
           return std::make_tuple(s, DynVec<real_t>(2, 0.0));
        };

```

## <a name="include_files"></a> Include files

```
#include "cubic_engine/base/cubic_engine_types.h"
#include "cubic_engine/search/rapidly_exploring_random_tree.h"
#include "kernel/utilities/csv_file_writer.h"
#include "kernel/dynamics/system_state.h"
#include "kernel/maths/constants.h"
#include "kernel/base/unit_converter.h"
#include "kernel/utilities/common_uitls.h"

#include <cmath>
#include <iostream>
#include <vector>
#include <random>
#include <string>
#include <fstream>

```

## <a name="m_func"></a> The main function

```
namespace example
{

using cengine::uint_t;
using cengine::real_t;
using cengine::DynMat;
using cengine::DynVec;
using cengine::search::RRT;
using kernel::dynamics::SysState;

const real_t TOL = 1.0e-8;
const real_t DT = 0.5;
const uint_t Nx = 50;
const uint_t Ny = 50;
const real_t Dx = 50.0/Nx;
const real_t xstart = 0.0;
const real_t Dy = 50.0/Ny;
const real_t ystart = 0.0;

typedef std::vector<SysState<2>> world_t;

// create the world
void create_world(world_t& world){

    world.resize(Nx*Ny);

    uint_t counter = 0;
    for(uint_t i =0; i<Nx; ++i){
        for(uint_t j=0; j<Ny; ++j){
          auto& state = world[counter++];
          state.set(0, {"X", xstart + i*Dx});
          state.set(1, {"Y", ystart + j*Dy});
        }
    }
}

void save_vertices(const RRT<SysState<2>, DynVec<real_t>>& rrt,
                   const std::string& filename){

    // open the file stream
    std::ofstream file;
    file.open(filename, std::ios_base::out);

    for(uint_t v=0; v<rrt.n_vertices(); ++v){
        auto& vertex = rrt.get_vertex(v);
        auto x = vertex.data.get("X");
        auto y = vertex.data.get("Y");
        file<<vertex.id<<","<<x<<","<<y<<std::endl;

    }
    file.close();
}

void save_connections(const RRT<SysState<2>, DynVec<real_t>>& rrt,
                      const std::string& filename){

    // open the file stream
    std::ofstream file;
    file.open(filename, std::ios_base::out);

    for(uint_t v=0; v<rrt.n_vertices(); ++v){

        auto vneighs = rrt.get_vertex_neighbors(v);
        auto& vertex = rrt.get_vertex(v);
        auto start = vneighs.first;
        auto end = vneighs.second;

        file<<vertex.id<<",";
        while(start != end){
            auto& neigh = rrt.get_vertex(start);

            if (boost::next(start) != end){
                    file<<neigh.id<<",";
            }
            else{
               file<<neigh.id<<"\n";
            }

            ++start;
        }
    }

    file.close();

}

void test_1(){

    world_t world;

    // create the nodes of the world
    create_world(world);

    std::cout<<"For test_1 world size: "<<world.size()<<std::endl;

    // how to select a state from the world
    auto state_selector = [&world](){

        std::random_device rd;
        std::mt19937 gen(rd());
        std::uniform_int_distribution<> distrib(0, world.size()-1);
        return world[distrib(gen)];
    };


    // compute the dynamics of the model. The tree simply
    // grows in the 45 degrees diagonal
    auto dynamics = [](const SysState<2>& s1, const SysState<2>& s2){
       SysState<2> s;
       s.set(0, {"X", 0.0});
       s.set(1, {"Y", 0.0});
       s["X"] = s1["X"] + 0.5;
       s["Y"] = s1["Y"] + 0.5;
       return std::make_tuple(s, DynVec<real_t>(2, 0.0));
    };

    // simply use l2 norm as the difference between
    // the two states
    auto metric = [](const SysState<2>& s1, const SysState<2>& s2 ){
        return l2Norm(s1.as_vector()-s2.as_vector());
    };

    // build the rrt
    SysState<2> xinit;
    xinit.set(0, {"X", 25.0});
    xinit.set(1, {"Y", 25.0});

    RRT<SysState<2>, DynVec<real_t>> rrt;
    rrt.set_show_iterations_flag(true);
    rrt.build(world.size(), xinit, state_selector, metric, dynamics);
    save_vertices(rrt, "rrt_test_1.txt");
    save_connections(rrt, "rrt_connections_test_1.txt");
}

void test_2(){

    world_t world;

    // create the nodes of the world
    create_world(world);

    std::cout<<"For test_1 world size: "<<world.size()<<std::endl;

    // how to select a state from the world
    auto state_selector = [&world](){

        std::random_device rd;
        std::mt19937 gen(rd());
        std::uniform_int_distribution<> distrib(0, world.size()-1);
        return world[distrib(gen)];
    };

    // compute the dynamics of the model.
    auto dynamics = [](const SysState<2>& s1, const SysState<2>& s2){
       SysState<2> s;
       s.set(0, {"X", 0.0});
       s.set(1, {"Y", 0.0});
       s["X"] = s1["X"] + (s2["X"] - s1["X"]);
       s["Y"] = s1["Y"] + (s2["Y"] - s1["Y"]);

       return std::make_tuple(s, DynVec<real_t>(2, 0.0));
    };

    // simply use l2 norm as the difference between
    // the two states
    auto metric = [](const SysState<2>& s1, const SysState<2>& s2 ){
        return l2Norm(s1.as_vector()-s2.as_vector());
    };

    // build the rrt
    SysState<2> xinit;
    xinit.set(0, {"X", 25.0});
    xinit.set(1, {"Y", 25.0});

    RRT<SysState<2>, DynVec<real_t>> rrt;
    rrt.set_show_iterations_flag(true);
    rrt.build(1000, xinit, state_selector, metric, dynamics);
    save_vertices(rrt, "rrt_test_2.txt");
    save_connections(rrt, "rrt_connections_test_2.txt");

}

}

int main() {
   
    using namespace example;

    try{

        //test_1();
        test_2();

    }
    catch(std::runtime_error& e){
        std::cerr<<"Runtime error: "
                 <<e.what()<<std::endl;
    }
    catch(std::logic_error& e){
        std::cerr<<"Logic error: "
                 <<e.what()<<std::endl;
    }
    catch(...){
        std::cerr<<"Unknown exception was raised whilst running simulation."<<std::endl;
    }
   
    return 0;
}

```

## <a name="results"></a> Results

As already mentioned in the introduction of this notebook, the first test case does not produce something very exciting.

<img src="line_rrt.png"
     alt="Line RRT"
     style="float: left; margin-right: 10px; width: 500px;" />

## <a name="source_code"></a> Source Code

<a href="../exe.cpp">exe.cpp</a>